In [1]:
import os
import glob

from struct import unpack
import shutil

import zipfile

from sklearn.model_selection import train_test_split

In [2]:
os.environ['KAGGLE_CONFIG_DIR'] = 'C:\\Users\\Tom\\Development\\azurecup-2021-triumviratus\\backend\\keychain\\kaggle'

In [3]:
%cd ../../data/image-database

C:\Users\Tom\Development\azurecup-2021-triumviratus\data\image-database


# Download the dataset

In [ ]:
local_zip = 'mushrooms-classification-common-genuss-images.zip'
if not os.path.exists(local_zip):
    os.makedirs('maysee-mushrooms-classification-common-genuss-images', exist_ok = True)
    %cd maysee-mushrooms-classification-common-genuss-images
    !kaggle datasets download -d maysee/mushrooms-classification-common-genuss-images

C:\Users\Tom\Development\azurecup-2021-triumviratus\data\image-database\maysee-mushrooms-classification-common-genuss-images


# Unzip data

In [ ]:
if not os.path.exists('Mushrooms'):
    with zipfile.ZipFile(local_zip,'r') as zip_ref:
        zip_ref.extractall('')

# Remove broken files

In [ ]:
marker_mapping = {
    0xffd8 : "Start of Image",
    0xffe0 : "Application Default Header",
    0xffdb : "Quantization Table",
    0xffc0 : "Start of Frame",
    0xffc4 : "Define Huffman Table",
    0xffda : "Start of Scan",
    0xffd9 : "End of Image"
}

class JPEG:
    def __init__(self, image_file):
        with open(image_file, 'rb') as f:
            self.img_data = f.read()
    
    def decode(self):
        data = self.img_data
        while(True):
            marker, = unpack(">H", data[0:2])

            if marker == 0xffd8:
                data = data[2:]
            elif marker == 0xffd9:
                return
            elif marker == 0xffda:
                data = data[-2:]
            else:
                lenchunk, = unpack(">H", data[2:4])
                data = data[2 + lenchunk:]            
            if len(data) == 0:
                break        


In [ ]:
bads = []

for img in glob.glob(os.path.join('Mushrooms/**','*')):
  image = JPEG(img)
  try:
    image.decode()   
  except:
    bads.append(img)

for name in bads:
  print('[X]',name)
  os.remove(name)

In [ ]:
%mkdir train
%mkdir test

In [ ]:
names = set()

for file in glob.glob(folder_glob):
  name = os.path.normpath(file).split(os.sep)[1]
  names.add(name)

for name in names:
  for set_name in ('train','test'):
    os.makedirs(os.path.join(set_name,name), exist_ok = True)


In [ ]:
train, test = train_test_split(glob.glob(folder_glob), test_size = 0.2)

for source in train:
  target = source.replace('Mushrooms', 'train')
  shutil.copy(source,target)

for source in test:
  target = source.replace('Mushrooms', 'test')
  shutil.copy(source,target)
